In [ ]:
import os
import gc
import shutil
import torch
from typing import List
from transformers import AutoTokenizer, GenerationConfig, AutoModelForCausalLM, AutoConfig
from peft import PeftConfig, PeftModel
from train import train

### Загрузка и сохранение на диск базовой модели.

In [2]:
adapter_path = 'models/adapter'
base_adapter_name = 'IlyaGusev/saiga2_7b_lora'

os.makedirs('models', exist_ok=True)
os.makedirs(adapter_path, exist_ok=True)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_adapter_name, use_fast=False, padding_side='left')
generation_config = GenerationConfig.from_pretrained(base_adapter_name, do_sample=True)

In [4]:
config = PeftConfig.from_pretrained(base_adapter_name)

In [5]:
base_model_name = config.base_model_name_or_path
base_model_config = AutoConfig.from_pretrained(base_model_name)

model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=base_model_config.torch_dtype,
    # load_in_8bit=True,
    # device_map="auto",
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:51<00:00, 25.68s/it]


In [6]:
model = PeftModel.from_pretrained(model, base_adapter_name)

In [ ]:
model.eval()

In [12]:
model.save_pretrained(adapter_path)
tokenizer.save_pretrained(adapter_path)
generation_config.save_pretrained(adapter_path)

model = None
gc.collect()
torch.cuda.empty_cache()

### Тренировка модели вопросах и ответах.

In [2]:
output_path = './models/recipes_model'
base_adapter_path = "./models/adapter"
os.makedirs(output_path, exist_ok=True)
config = PeftConfig.from_pretrained(base_adapter_path)
base_model_name = config.base_model_name_or_path

In [ ]:
train(config_file='config/train_recipes_config.json',
     train_file='datasets/train_recipes.jsonl',
     val_file='datasets/test_recipes.jsonl',
     model_path=base_model_name,
     adapter_path=base_adapter_path, 
     report_to='none',
     output_dir=output_path
     )

deepspeed:  None
models/llama2-7b


Vocab size:  32000
PAD:  0 <unk>
BOS:  1 <s>
EOS:  2 </s>
UNK:  0 <unk>
SEP:  None None


100%|██████████| 1500/1500 [00:03<00:00, 399.58it/s]


Load model
use 8 bit: True
use 4 bit: False


Loading checkpoint shards: 100%|██████████| 2/2 [00:19<00:00, 10.00s/it]


Load Adapter


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


training run.


Step,Training Loss,Validation Loss
5,1.752100,1.680979
10,1.597800,1.580341
15,1.534800,1.509741


### Тренировка модели на диалогах с персонажем.

Происходит дообучение адаптера из предыдущего этапа.

In [2]:
output_path = './models/granny_model'
base_adapter_path="./models/recipes_model"
os.makedirs(output_path, exist_ok=True)
config = PeftConfig.from_pretrained(base_adapter_path)
base_model_name = config.base_model_name_or_path

In [3]:
train(config_file="config/train_granny_config.json",
     train_file='datasets/train_dialogues.jsonl',
     val_file='datasets/test_dialogues.jsonl',
     model_path=base_model_name,
     adapter_path=base_adapter_path, 
     report_to='none',
     output_dir=output_path
     )

deepspeed:  None
models/llama2-7b


Vocab size:  32000
PAD:  0 <unk>
BOS:  1 <s>
EOS:  2 </s>
UNK:  0 <unk>
SEP:  None None


100%|██████████| 3/3 [00:00<00:00, 252.07it/s]

Load model
use 8 bit: True
use 4 bit: False



Loading checkpoint shards: 100%|██████████| 2/2 [01:13<00:00, 36.80s/it]


Load Adapter


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


training run.


Step,Training Loss,Validation Loss
5,No log,0.988845
10,0.117200,0.902335
15,0.117200,0.880322
20,0.085500,0.923268
25,0.085500,0.964853
30,0.048500,1.079148
35,0.048500,1.175153
40,0.015000,1.322826
45,0.015000,1.412606
50,0.002800,1.567160


save model to ./models/granny_model


### Запуск модели внутри сервиса.

Для того чтобы сервис подхватил модель, ее необходимо переместить из папки 
./train/models/granny_model в папку models.

In [7]:
os.makedirs('../models', exist_ok=True)
file_dir = os.path.dirname(os.path.realpath('__file__'))

source_dir = os.path.join(file_dir, 'models/granny_model')
dst_dir = os.path.join(file_dir, '../models')

In [ ]:
shutil.copytree(source_dir, dst_dir, dirs_exist_ok=True, symlinks=True)